TO DO: 
  - set up per metric instead of wer metric
  - make transformer code more efficient
  - get all the data needed
  - train model
  - build UI (app maybe for experimentation)

In [1]:
from huggingface_hub import notebook_login

notebook_login()

In [4]:
# import statements
from interface.TimitInterface import TimitInterface # for getting the data file names in necessary format
from phoneme import extract_needed_data # for retriveing necessary information from data file location
from transformers import Wav2Vec2CTCTokenizer, Wav2Vec2FeatureExtractor, Wav2Vec2Processor, Wav2Vec2Config, Wav2Vec2ForCTC # needed to run wav2vec2 model
from transformers import TrainingArguments, Trainer # needed to train model
from datasets import Dataset, load_metric # to convert extracted data into the ideal format needed for model training; load_metric needed to load wer metric
import numpy as np
import torch

In [5]:
dataset = TimitInterface()

0it [00:00, ?it/s]

Map:   0%|          | 0/2688 [00:00<?, ? examples/s]

Map:   0%|          | 0/336 [00:00<?, ? examples/s]

Map:   0%|          | 0/336 [00:00<?, ? examples/s]

In [6]:
all_data = dataset.data
train_data = dataset.train
test_data = dataset.test
val_data = dataset.valid

In [7]:
train_data_keys = list(train_data.keys())
test_data_keys = list(test_data.keys())
val_data_keys = list(val_data.keys())

In [8]:
# from phoneme import extract_needed_data

In [10]:
# from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer("./interface/phoneme.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")
#tokenizer = Wav2Vec2CTCTokenizer("./phoneme.json", unk_token="[UNK]", pad_token="[PAD]")

c:\Users\kavya\anaconda3\envs\accent-capstone\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [63]:
repo_name = "asr-for-phoneme-detection-3"
token = "" #insert your token from huggingface here

In [64]:
from huggingface_hub import HfApi

# Initialize the API client
api = HfApi()

# Create the repository
api.create_repo(
    repo_id=repo_name,
    token=token,
    exist_ok=True
)

RepoUrl('https://huggingface.co/kavsss/wav2vec2-tune-timit-asr-for-phoneme-2', endpoint='https://huggingface.co', repo_type='model', repo_id='kavsss/wav2vec2-tune-timit-asr-for-phoneme-2')

In [65]:
tokenizer.push_to_hub(repo_name)

CommitInfo(commit_url='https://huggingface.co/kavsss/wav2vec2-tune-timit-asr-for-phoneme-2/commit/776edbddbf815c286eac58a5127ecb9e2e28d924', commit_message='Upload tokenizer', commit_description='', oid='776edbddbf815c286eac58a5127ecb9e2e28d924', pr_url=None, pr_revision=None, pr_num=None)

In [14]:
# from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=False)

In [15]:
# from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [16]:
def prepare_dataset(batch):
    #audio = batch["audio"]

    # batched output is "un-batched" to ensure mapping is correct
    batch["input_values"] = processor(batch["audio_arr"], sampling_rate=16000).input_values[0]
    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["phoneme"]).input_ids
    return batch

Code to check that the data is in the correct format

In [18]:
path = './data/'+train_data_keys[0]
print(path+'.WAV')
audio_arr, line, phoneme = extract_needed_data(path)
batch = {"audio_arr": audio_arr, "line": line, "phoneme": phoneme}
batch = prepare_dataset(batch)

./data/TEST/DR7/FISB0/SA1.WAV


c:\Users\kavya\anaconda3\envs\accent-capstone\lib\site-packages\transformers\models\wav2vec2\processing_wav2vec2.py:157: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(


In [20]:
'./data/'+val_data_keys[0]

'./data/TRAIN/DR3/MKXL0/SX195'

In [21]:
audio_arr

array([-0.00032773,  0.00038493,  0.00050656, ...,  0.00165166,
       -0.00132316, -0.00016492], dtype=float32)

In [22]:
line

'She had your dark suit in greasy wash water all year.'

In [23]:
phoneme

'sh|iy|hv|ae|dcl|d|y|er|dcl|d|aa|r|kcl|k|s|ux|tcl|ih|n|gcl|g|r|iy|z|iy|w|ao|sh|epi|w|ao|dx|er|q|ao|l|y|ih|axr'

In [24]:
batch["input_values"]

array([-0.0027629 ,  0.00404618,  0.00520826, ...,  0.01614906,
       -0.01227374, -0.00120741], dtype=float32)

In [25]:
batch["labels"]

[50,
 0,
 33,
 0,
 30,
 0,
 2,
 0,
 14,
 0,
 13,
 0,
 59,
 0,
 23,
 0,
 14,
 0,
 13,
 0,
 1,
 0,
 48,
 0,
 36,
 0,
 35,
 0,
 49,
 0,
 56,
 0,
 52,
 0,
 31,
 0,
 39,
 0,
 27,
 0,
 26,
 0,
 48,
 0,
 33,
 0,
 60,
 0,
 33,
 0,
 58,
 0,
 4,
 0,
 50,
 0,
 22,
 0,
 58,
 0,
 4,
 0,
 16,
 0,
 23,
 0,
 47,
 0,
 4,
 0,
 37,
 0,
 59,
 0,
 31,
 0,
 8]

In [27]:
path = './data/TEST/DR3/MBWM0/SX134'
print(path+'.WAV')
audio_arr, line, phoneme = extract_needed_data(path)
batch = {"audio_arr": audio_arr, "line": line, "phoneme": phoneme}
batch = prepare_dataset(batch)

./data/TEST/DR3/MBWM0/SX134.WAV


c:\Users\kavya\anaconda3\envs\accent-capstone\lib\site-packages\transformers\models\wav2vec2\processing_wav2vec2.py:157: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(


In [28]:
phoneme

'd|ix|s|eh|m|bcl|b|axr|ix|n|dcl|jh|ae|n|y|ix|w|axr|ih|er|n|ay|s|epi|m|ah|n|t|s|tcl|t|ax-h|s|pcl|p|eh|n|dcl|ix|n|m|ay|ae|m|iy'

In [29]:
# from datasets import Dataset

In [30]:
input_values = []
labels = []
for t_path in train_data_keys:
    path = './data/'+t_path 
    audio_arr, line, phoneme = extract_needed_data(path)
    batch = {"audio_arr": audio_arr, "line": line, "phoneme": phoneme}
    batch = prepare_dataset(batch)
    #timit_train["features"]["input_values"].append(batch["input_values"])
    #timit_train["features"]["labels"].append(batch["labels"])
    input_values.append(batch["input_values"])
    labels.append(batch["labels"])

In [31]:
# Create a Dataset from the lists
train_dataset = Dataset.from_dict({
    "input_values": input_values,
    "labels": labels
})

In [34]:
input_values = []
labels = []
for t_path in test_data_keys:
    path = './data/'+t_path 
    audio_arr, line, phoneme = extract_needed_data(path)
    batch = {"audio_arr": audio_arr, "line": line, "phoneme": phoneme}
    batch = prepare_dataset(batch)
    #timit_test["features"]["input_values"].append(batch["input_values"])
    #timit_test["features"]["labels"].append(batch["labels"])
    input_values.append(batch["input_values"])
    labels.append(batch["labels"])

In [35]:
test_dataset = Dataset.from_dict({
    "input_values": input_values,
    "labels": labels
})

In [36]:
input_values = []
labels = []
for t_path in val_data_keys:
    path = './data/'+t_path 
    audio_arr, line, phoneme = extract_needed_data(path)
    batch = {"audio_arr": audio_arr, "line": line, "phoneme": phoneme}
    batch = prepare_dataset(batch)
    #timit_val["features"]["input_values"].append(batch["input_values"])
    #timit_val["features"]["labels"].append(batch["labels"])
    input_values.append(batch["input_values"])
    labels.append(batch["labels"])

In [37]:
val_dataset = Dataset.from_dict({
    "input_values": input_values,
    "labels": labels
})

In [38]:
# import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [39]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [40]:
# from datasets import load_metric

wer_metric = load_metric("wer", trust_remote_code=True)
# a little unsure if wer is working
# wer is not working, we need per metric

C:\Users\kavya\AppData\Local\Temp\ipykernel_5972\4113679689.py:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  wer_metric = load_metric("wer", trust_remote_code=True)


In [41]:
# import numpy as np
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [71]:
# from transformers import Wav2Vec2Config, Wav2Vec2ForCTC

# Load your current model config
# config = Wav2Vec2Config.from_pretrained("facebook/wav2vec2-base")
config = Wav2Vec2Config.from_pretrained("kavsss/asr-for-phoneme-detection-2")

# Update the vocab_size
config.vocab_size = 64 # sinze there are 61 phonemes sounds and 3 special tokens on top of that
config.ctc_loss_reduction = "mean"
config.pad_token_id=processor.tokenizer.pad_token_id

# Save the updated config and load the model
config.save_pretrained("./asr-for-phoneme-detection-3")
# model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base", config=config)
model = Wav2Vec2ForCTC.from_pretrained("kavsss/asr-for-phoneme-detection-2", config=config)

Some weights of the model checkpoint at kavsss/wav2vec2-tune-timit-asr-for-phoneme were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at kavsss/wav2vec2-tune-timit-asr-for-phoneme and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You sho

In [72]:
model.freeze_feature_extractor()

In [73]:
# from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir=repo_name,
  group_by_length=True,
  per_device_train_batch_size=4, #og: 32
  evaluation_strategy="steps",
  num_train_epochs=4,#og: 30
  fp16=False,
  gradient_checkpointing=True, 
  save_steps=500,
  eval_steps=500,
  logging_steps=500,
  learning_rate=1e-4,
  #no_cuda=True,
  weight_decay=0.005,
  warmup_steps=1000,
  save_total_limit=2,
  push_to_hub=True
)

In [74]:
# from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=processor.feature_extractor,
)

In [75]:
trainer.train()

  0%|          | 0/2688 [00:00<?, ?it/s]

c:\Users\kavya\anaconda3\envs\accent-capstone\lib\site-packages\transformers\models\wav2vec2\processing_wav2vec2.py:157: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
c:\Users\kavya\anaconda3\envs\accent-capstone\lib\site-packages\torch\_dynamo\eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
c:\Users\kavya\anaconda3\envs\accent-capstone\lib\site-packages\torch\utils\checkpoint.py:295: FutureWarning: `to

{'loss': 2.7602, 'grad_norm': 10.608319282531738, 'learning_rate': 5e-05, 'epoch': 0.74}


  0%|          | 0/42 [00:00<?, ?it/s]

{'eval_loss': 0.8088061213493347, 'eval_wer': 0.3476037284500536, 'eval_runtime': 173.6748, 'eval_samples_per_second': 1.935, 'eval_steps_per_second': 0.242, 'epoch': 0.74}


...

  0%|          | 0/42 [00:00<?, ?it/s]

{'eval_loss': 0.2475772202014923, 'eval_wer': 0.13808463251670378, 'eval_runtime': 119.8119, 'eval_samples_per_second': 2.804, 'eval_steps_per_second': 0.351, 'epoch': 3.72}


c:\Users\kavya\anaconda3\envs\accent-capstone\lib\site-packages\transformers\models\wav2vec2\processing_wav2vec2.py:157: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
c:\Users\kavya\anaconda3\envs\accent-capstone\lib\site-packages\torch\_dynamo\eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
c:\Users\kavya\anaconda3\envs\accent-capstone\lib\site-packages\torch\utils\checkpoint.py:295: FutureWarning: `to

{'train_runtime': 11788.4144, 'train_samples_per_second': 0.912, 'train_steps_per_second': 0.228, 'train_loss': 0.8129628811563764, 'epoch': 4.0}


TrainOutput(global_step=2688, training_loss=0.8129628811563764, metrics={'train_runtime': 11788.4144, 'train_samples_per_second': 0.912, 'train_steps_per_second': 0.228, 'total_flos': 3.037705722038615e+17, 'train_loss': 0.8129628811563764, 'epoch': 4.0})

In [76]:
trainer.push_to_hub(repo_name)#, commit_message="training model")

CommitInfo(commit_url='https://huggingface.co/kavsss/wav2vec2-tune-timit-asr-for-phoneme-2/commit/8eee5ea621930d81c702260c5171f42bce317ee9', commit_message='wav2vec2-tune-timit-asr-for-phoneme-2', commit_description='', oid='8eee5ea621930d81c702260c5171f42bce317ee9', pr_url=None, pr_revision=None, pr_num=None)